# Problem 1: Numerical solution of the steady-state cable equation

We are supposed to solve the steady cable equation with an injected current at the centre of the cable. In order to get close to the case of an infinite cable we use the sealed end condition. That is in our case V'(-5) = V'(5) = 0.

In [ ]:
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from scipy import integrate, arange

# Plots will be done in the notebook:
%matplotlib inline

First we define the steady state cable equation. However we consider an injected current on an 0.2 mm interval around the point 0 instead of current that is precisely injected at 0. A smaller interval would lead to unwanted numerical effects, no matter how small we choose the spatial grid.

Furthermore we calculate the electronic lenght constant $\lambda$ and B := V(0) from our analytical solution to be able to compare it with the numerical solution later.

In [ ]:
I_e = 10 # nA
r_m = 2 # Ohm m
a = 2 # micro m
r_L = 2 # Ohm m
E_L = 0

lamb = np.sqrt(a*r_m/(2*r_L)) # m
B = I_e/2*r_m/(2*np.pi*a*lamb) # V

def steady_cable(y,x):
    
    # notations
    V = y[0]
    dVdx = y[1]
    
    # output vector
    dydx = np.zeros(2)
    
    # Here we determine the input current density
    # on an interval of lengh 0.2 mm around the point 0 (shorter not possible).
    # We use this current density to approximate the case
    # of an current injection at the point 0.
    if x >= -0.1 and x <= 0.1 : # mm
        i_e = I_e/(2*np.pi*a*0.2)
    else:
        i_e = 0
    
    # ODEs
    dydx[0] = dVdx
    dydx[1] = 2*r_L/a*((V-E_L)/r_m-i_e) 
    
    return dydx
    

Now we integrate. We consider a cable of 1 cm length. We choose the initial value V(-5) = 0.0107 which could be close to the correct one.

In [ ]:
x = np.linspace(-5,5,10001)
y_start = (0.0107, 0)
y = integrate.odeint(steady_cable, y_start, x)
V = y[:,0]

# plot
plt.figure()
plt.title('Steady-state voltage along the cable')
plt.ylabel('V [V]')
plt.xlabel('x [mm]')
plt.plot(x, V)

As we can see the sealed end condition V'(-5) = V'(5) = 0 is only satisfied on the left side, where we set it as a part of the initial condition. Thus we try a slightly larger initial value V(-5) = 0.0108. 

In [ ]:
y_start = (0.0108, 0)
y = integrate.odeint(steady_cable, y_start, x)
V = y[:,0]

# plot
plt.figure()
plt.title('Steady-state voltage along the cable')
plt.ylabel('V [V]')
plt.xlabel('x [mm]')
plt.plot(x, V)

Again the sealed end condition is not satisfied. This time the initial value is to big. In order to find the correct initial value we use the shooting method. We try different intial values and increase resp. decrease them till we obtain a solution that satisfies our boundary conditions.

In [ ]:
# Now we want to do the same with the shooting method

# first initial value for the voltage at -2
# for the slope we can always take 0, because of our sealed end condition
V_start = 1

# value in order to start the while loop
slope_end = 1

for n in range (1,21):
    if slope_end > 0:
        # If we start with a too large value for V_start
        # then the slope of V will be larger than than 0 at x=5.
        # So we must decrease the initial value till slope_end is smaller than 0.
        while slope_end > 0:
            # redifine initial value:
            V_start = V_start*(1 - 2**(-n))
            y = integrate.odeint(steady_cable, (V_start, 0), x)
            slope_end = y[-1,1]
    else:
        # If we have a too small value for V_start
        # then the slope of V will be smaller than than 0 at x=5.
        # So we must increase V_start till slope_end is larger than 0.
        while slope_end < 0:
            # redifine initial value:
            V_start = V_start*(1 + 2**(-n))
            y = integrate.odeint(steady_cable, (V_start, 0), x)
            slope_end = y[-1,1]            
        # Since 2**(-n) becomes smaller and smaller, we get closer to the correct solution.
V = y[:,0]
print 'final initial condidtion:', V_start


# matplotlib.rcParams.update({'font.size': 24})


# plot
fig = plt.figure(figsize=(15,10))
plt.title('Steady-state voltage along the cable')
plt.ylabel('V [V]')
plt.xlabel('x [mm]')
plt.plot(x, V)
# fig.savefig('num_sol.png')

This time the boundary conditions are satisfied. We compare this solution with our analytical result now. 

In [ ]:
def analytic_solution_scalar(x):
    return B*np.exp(-np.absolute(x)/lamb)
analytic_solution = np.vectorize(analytic_solution_scalar)

V_analytical = analytic_solution(x)

fig = plt.figure(figsize=(15,10))
plt.title('Steady-state voltage along the cable')
plt.ylabel('V [V]')
plt.xlabel('x [mm]')
plt.plot(x, V, color="blue", label="numerical solution")
plt.plot(x, V_analytical, color="red", label="analytical solution")
plt.legend()
#fig.savefig('comp.png')

We observe that our numerical solution is a bit inaccurate around zero and near the ends of the cable. However this is not a big surprise. After all we simulated a 1 cm long cable with an current injected by an 0.2 mm thick electrode, but we calculated the analytical solution for an infinite cable and a current injected exactly at one point.

In total our numerical solution is accurate enough.

# Problems 2-6: Multi-Compartment neuron

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 22})
from multicompartment import MultiCompartmentHodgkinHuxley
import numpy as np
np.seterr(all='warn')

In [ ]:
# exercise 3
# fig 6.17 
m = MultiCompartmentHodgkinHuxley(N=201, Ie=[2e-6], mu_inj=[0], L=3e-2)
m.solve(t=0.2, dt=0.00005)

In [ ]:
mu = [50,150]
mu_length = m.L/m.N * 1000. # in mm
fig,ax = plt.subplots(3,1,figsize=(30,20))
for i,ti in enumerate([600]):#np.arange(0,2000,100)):#[0, 1, 50, 100, 300, 500, 800, 999]):
    ax[0].plot(np.arange(m.N)*mu_length, m.v[:,ti]*1000., label='$t = {}$ ms'.format(m.dt*ti*1000.), linewidth=2)
ax[0].legend()
ax[0].set_xlabel('$x$ [mm]')
ax[0].set_ylabel('$V$ [mV]')
ax[0].vlines(mu[0]*mu_length, -80., m.v[mu[0],600]*1000., color='r', linewidth=2)
ax[0].vlines(mu[1]*mu_length, -80., m.v[mu[1],600]*1000., color='g', linewidth=2)
# mu=[50,199]
ax[1].plot(m.t*1000, m.v[mu[0],:]*1000., c='r', label='$\mu = {}$ at $x = {:.2f}$ mm'.format(mu[0],mu[0]*mu_length), linewidth=2)
ax[2].plot(m.t*1000, m.v[mu[1],:]*1000., c='g', label='$\mu = {}$ at $x = {:.2f}$ mm'.format(mu[1],mu[1]*mu_length), linewidth=2)
ax[1].set_xlim(26,34)
ax[2].set_xlim(26,34)
ax[1].set_xlabel('$t$ [ms]')
ax[2].set_xlabel('$t$ [ms]')
ax[1].set_ylabel('$V$ [mV]')
ax[2].set_ylabel('$V$ [mV]')
ax[1].legend()
ax[2].legend()
# ax[1].vlines(30, -80, m.v[mu[0],600])
plt.savefig('ex3.png')
#4
m.calculate_propagation_velocity(50,199)
print 'velocity is', m.prop_velocity
print m.v[mu[0],600]*1000
print mu[0]*mu_length

In [ ]:
mu = [0]
fig,ax = plt.subplots(1,1,figsize=(15,10))
ax.plot(m.t*1000, m.v[mu[0],:]*1000., c='r', label='$\mu = {}$ at $x = {:.2f}$ mm'.format(mu[0],mu[0]*mu_length), linewidth=2)
# ax[1].set_xlim(26,34)
# ax[2].set_xlim(26,34)
ax.set_xlabel('$t$ [ms]')
ax.set_ylabel('$V$ [mV]')
ax.set_title('Constant current injection at $x = 0$')
ax.legend()
plt.savefig('ex3_1.png')

In [ ]:
# ex 4
A = np.arange(11,90,5)*1e-6
vel = np.zeros(A.size)
anal_vel = np.zeros(A.size)
for i,a in enumerate(A):
    print ('calculating a =', a)
    m = MultiCompartmentHodgkinHuxley(N=201, Ie=[(7+i)*1e-8], mu_inj=[0], L=3e-2, a=a)
    m.solve(t=0.1, dt=0.0001)
    m.calculate_propagation_velocity(50,150,test=1)
    vel[i] = m.prop_velocity
    anal_vel[i] = m.anal_vel

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(A*1e6, vel,label='$\Delta x / \Delta t$', linewidth=3)
plt.plot(A*1e6, np.pi*anal_vel, label='$\pi \cdot \lambda / \\tau _m$',color='r', linewidth=3)
#plt.plot(A*1e6, vel/anal_vel)
plt.legend(loc=4,prop={'size':28})
plt.ylabel('velocity [m/s]')
plt.xlabel('a [$\mu$m]')
plt.title('Action potential propagation velocity')

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(m.t*1e3, m.v[50,:]*1e3, label='comp. 50/200', linewidth=3)
plt.plot(m.t*1e3, m.v[150,:]*1e3, label='comp. 150/200', linewidth=3)
ax = plt.gca()
ax.set_xlim(14,20)
plt.ylabel('V [mV]')
plt.xlabel('t [ms]')
plt.legend(prop={'size':16})
plt.title('')
ax.vlines([15.57, 16.98],-80,40, color='k',linestyle=':', linewidth=2, alpha=0.5)
ax.hlines(0, 15.57, 16.98, color='k',linewidth=3, alpha=0.5)
ax.text(15.8, 1, '$\Delta t$', fontsize=20)

In [ ]:
#ex 5
m = MultiCompartmentHodgkinHuxley(N=201, Ie=[2e-6,2e-6], mu_inj=[0,200], L=10e-2)
m.solve(t=2, dt=0.0001)

In [ ]:
fig, ax = plt.subplots(5,1,figsize=(25,15))
num = m.t/m.dt
mid = num/2
# col=['#3300CC','#6600AA','#990099', '#AA0066','#CC0033']
# style=['-', '--', ':', '-.','--']
for i,ti in enumerate([20,40,60,80,90]):
    ax[i].plot(np.arange(m.N)*m.L/m.N*1000, m.v[:,ti]*1e3, color='k',label='t = {} ms'.format(np.round(m.t[ti]*1e3,1)), linewidth=2)
    ax[i].legend()
    ax[i].set_ylabel('$V$ [mV]')
#     ax[i].set_xlabel('t [ms]')
    ax[i].set_ylim(-80,45)
ax[-1].set_xlabel('$x$ [mm]')
plt.setp([a.get_xticklabels() for a in ax[:-1]], visible=False)
plt.savefig('ex5.png')

### The following did not get any usefull results

In [ ]:
#6
m = MultiCompartmentHodgkinHuxley(N=5000, Ie=[1e-3], mu_inj=[500], L=5e-3, myelinated=True)
m.solve(t=0.01, dt=0.00001)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(30,10))
for i,ti in enumerate([0,1,20,100,200,500,998]):
    ax.plot(range(m.N), m.v[:,ti], label='$t$ = {} ms'.format(m.t[ti]*1e3))
plt.legend()
ax.set_xlabel('$\mu$')
ax.set_ylabel('V [V]')
col = np.where(m.myelin_pos!=True)
# ax.vlines(col,-0.07,0.07, colors='r', alpha=0.5)
print col
# ax.set_xlim(450,550)

In [ ]:
plt.plot(m.t, m.v[505,:])